In [ ]:
from collections import defaultdict

In [118]:
with open("./input5.dat", "r") as f:
    lines = f.readlines()
dd = defaultdict(list)
attr_name = 'seeds'
dd[attr_name] = [int(s) for s in lines[0].split(":")[1].split()]
range_list = []
for line in lines[2:]:
    if line=="\n": #end of map
        # print(range_list)
        #use range_list if in range_list, apply rule, else else else keep value
        dd[attr_name] = []
        for el in dd[prev_attr_name]:
            for i in range(len(range_list)):
                [post, pre, length] = range_list[i] #as int
                if el >=pre and el<pre+length:
                    dd[attr_name].append(post+el-pre)
                    break
            else:
                dd[attr_name].append(el)
        continue
    if line[-5:-1]=="map:": #begin of map
        range_list = []
        prev_attr_name = attr_name
        attr_name = line.split()[0].split("-")[2]
        # map_names.append(line.split()[0])#.split("-")
        # print(line)
    else:
        [post, pre, length] = line.split() #as strings
        range_list.append([int(post), int(pre), int(length)])
#one last time because the input doesn't end in "\n"
dd[attr_name] = []
for el in dd[prev_attr_name]:
    for i in range(len(range_list)):
        [post, pre, length] = range_list[i] #as int
        if el >=pre and el<pre+length:
            dd[attr_name].append(post+el-pre)
            break
    else:
        dd[attr_name].append(el)
min(dd['location'])

457535844

In [117]:
dd.keys(), 
for attr in dd.keys():
    print(attr+": min item = "+str(min(dd[attr])))

seeds: min item = 8403417
soil: min item = 41165510
fertilizer: min item = 202599792
water: min item = 368331544
light: min item = 159150094
temperature: min item = 75017753
humidity: min item = 640121624
location: min item = 457535844


In [150]:
#works in theory, but too long for big file
with open("./exampleinput5.dat", "r") as f:
    lines = f.readlines()
lines.append("\n") #because the input doesn't end in "\n"
dd = defaultdict(list)
attr_name = 'seeds'
prelist = [int(s) for s in lines[0].split(":")[1].split()]
dd[attr_name] = [_ for base, dist in zip(prelist[0::2], prelist[1::2]) for _ in range(base,base+dist)]
range_list = []
for line in lines[2:]:
    if line=="\n": #end of map
        #use range_list if in range_list, apply rule, else else else keep value
        dd[attr_name] = []
        for el in dd[prev_attr_name]:
            for i in range(len(range_list)):
                [post, pre, length] = range_list[i] #as int
                if el >=pre and el<pre+length:
                    dd[attr_name].append(post+el-pre)
                    break
            else:
                dd[attr_name].append(el)
        continue
    if line[-5:-1]=="map:": #begin of map
        range_list = []
        prev_attr_name = attr_name
        attr_name = line.split()[0].split("-")[2]
        # map_names.append(line.split()[0])#.split("-")
        # print(line)
    else:
        [post, pre, length] = line.split() #as strings
        range_list.append([int(post), int(pre), int(length)])

import numpy as np
pos_min = np.argmin(dd['location'])
for attr in dd.keys():
    print(attr+": min-chain item = "+str(dd[attr][pos_min]))
print("min of location is", min(dd['location']))

seeds: min-chain item = 82
soil: min-chain item = 84
fertilizer: min-chain item = 84
water: min-chain item = 84
light: min-chain item = 77
temperature: min-chain item = 45
humidity: min-chain item = 46
location: min-chain item = 46
min of location is 46


In [254]:
with open("./input5.dat", "r") as f:
    lines = f.readlines()
lines.append("\n") #because the input doesn't end in "\n"
min_location = 0
dd = defaultdict(list)
attr_name = 'seed'
prelist = [int(s) for s in lines[0].split(":")[1].split()]
dd[attr_name] = [[base, base+dist] for base, dist in zip(prelist[0::2], prelist[1::2])]
# dd[attr_name] = zip(prelist[0], prelist[0]+prelist[1]) #single range of seeds
range_list = []
for line in lines[2:]:
    if line[-5:-1]=="map:": #begin of map
        range_list = []
        prev_attr_name = attr_name
        attr_name = line.split()[0].split("-")[2]
        if prev_attr_name != line.split()[0].split("-")[0]:
            raise ValueError("Previous attribute name no matched", prev_attr_name, attr_name, line)
        # print(line)
    elif len(line)>1:
        [post, pre, length] = line.split() #as strings
        range_list.append([int(post), int(pre), int(length)])

    else:#if line=="\n": #end of map
        dd[attr_name] = []
        for prev_low, prev_high in dd[prev_attr_name]:
            # prev_range = set(range(prev_low, prev_high)) #range intersection is slow. Use extermes only!
            translated = [] #instances that have been translated to next attribute
            for i in range(len(range_list)):
                [post, pre, length] = range_list[i] #as int
                # intersect = prev_range.intersection(set(range(pre, pre+length)))
                elemlow, elemhigh = max(prev_low, pre), min(prev_high, pre+length) #max of mins, min of maxs between two intervals to get intersection
                if elemlow < elemhigh:
                    # elemlow, elemhigh = min(intersect), max(intersect)+1  #pythonic +1 at end of range
                    translated.append([elemlow, elemhigh])
                    dd[attr_name].append([post+elemlow-pre, post+elemhigh-pre])
            #sort and merge in translated
            translated.sort()
            i = 0
            while i < len(translated)-1:
                if translated[i][1] == translated[i+1][0]: #equality bc pythonic range limits
                    translated[i] = [translated[i][0], translated[i+1][1]]
                    translated.remove(translated[i+1])
                    continue #don't add 1, need to test joined with next
                i = i+1
            #add leftovers (untranslated) as identity    
            if translated == []:
                dd[attr_name].append([prev_low, prev_high])
                continue
            if translated[0][0] > prev_low:
                dd[attr_name].append([prev_low, translated[0][0]])
            for i in range(len(translated)-1):
                dd[attr_name].append([translated[i][-1],translated[i+1][0]])
            if translated[-1][-1] < prev_high:
                dd[attr_name].append([translated[-1][-1], prev_high])

        #join adjacent ranges
        dd[attr_name].sort()
        i = 0
        while i < len(dd[attr_name])-1: #while list not finished (possibly early bc of removals)
            if dd[attr_name][i][1] == dd[attr_name][i+1][0]: #equality bc pythonic range limits
                dd[attr_name][i] = [dd[attr_name][i][0], dd[attr_name][i+1][1]]
                dd[attr_name].remove(dd[attr_name][i+1])
                continue #don't add 1, need to test joined with next
            i = i+1

#print(dd)
print("min of location is", min(dd['location'])[0])

min of location is 41222968


In [256]:
dd['location']

[[41222968, 42024284],
 [68608231, 71836547],
 [120247692, 121678607],
 [154835034, 174272382],
 [186829220, 227494383],
 [337210449, 348953791],
 [374784574, 395248650],
 [416809660, 449132427],
 [467364259, 475007602],
 [475876376, 500368604],
 [504661783, 531676870],
 [551720858, 578799325],
 [599623118, 631556933],
 [647749131, 711128092],
 [776541253, 779681654],
 [800398694, 805990732],
 [872467855, 889502516],
 [1017520276, 1056868676],
 [1081323768, 1135055763],
 [1148776330, 1151510194],
 [1169583301, 1181910625],
 [1235328434, 1290798890],
 [1310911994, 1312650463],
 [1354405294, 1365411120],
 [1387390964, 1495209180],
 [1659783208, 1665059008],
 [1767145617, 1849246768],
 [1974296651, 1996039366],
 [2003624614, 2005920830],
 [2008996707, 2015503773],
 [2057694166, 2097696890],
 [2133436297, 2158819357],
 [2181345242, 2186686145],
 [2245504116, 2273766396],
 [2293269716, 2293415371],
 [2353465194, 2362963902],
 [2375889935, 2409771453],
 [2467817302, 2482710318],
 [2543777270